In [38]:
# Standard Libraries
import pandas as pd
import numpy as np
import os
import sys
from datetime import date
import shutil
import requests
from PIL import Image
import pickle
sys.path.insert(0, os.path.abspath('../src'))

# Custom Libraries
from file_proc import *
#import database

# Machine Learning
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import datasets, preprocessing, feature_extraction, linear_model, svm, metrics, ensemble, tree, ensemble
from sklearn.metrics import log_loss, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

# Image Libraries
import cv2
from cv2 import imread, imshow, waitKey, destroyAllWindows, rectangle, CascadeClassifier

# Neural Network Libraries
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

# Database
import psycopg2

# Visualizations
import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# import importlib
# importlib.reload(some_module)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Get the classifier .xml files and save them in the src directory
! curl -o ../src/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
! curl -o ../src/haarcascade_smile.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_smile.xml
! curl -o ../src/haarcascade_eye.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
! curl -o ../src/haarcascade_eye_tree_eyeglasses.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye_tree_eyeglasses.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  908k  100  908k    0     0  1518k      0 --:--:-- --:--:-- --:--:-- 1521k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0   605k      0 --:--:-- --:--:-- --:--:--  603k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  333k  100  333k    0     0   872k      0 --:--:-- --:--:-- --:--:--  872k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  587k  100  587k    0     0  1502k      0 --:--:-- --:--:-- --:--:-- 1498k


# Database

In [5]:
def test_database(db='kennedy', user_id='postgres', passwd=''):
    
    con = None

    try:
    
        con = psycopg2.connect(database=db, user=user_id,
            password=passwd)

        cur = con.cursor()
        cur.execute('SELECT version()')
        version = cur.fetchone()[0]
        print(version)

    except psycopg2.DatabaseError as e:

        print('Error {}'.format(e))
        sys.exit(1)

    finally:

        if con:
            con.close()
            
    return

In [6]:
def connect_database(db='kennedy', user_id='postgres', passwd=''):
    
    con = None

    try:
    
        con = psycopg2.connect(database=db, user=user_id,
            password=passwd)
        cur = con.cursor()
        
    except psycopg2.DatabaseError as e:
        
        print('Error {}'.format(e))
        sys.exit(1)
        
    return con, cur

In [7]:
def close_database(con):
    
    if con:
        con.commit()
        con.close()
        
    return 1

In [8]:
def create_database(con, cur):

        with con:

            cur.execute("DROP TABLE IF EXISTS images")
            cur.execute("CREATE TABLE images(id BIGINT DEFAULT 0 NOT NULL, file_name TEXT, dir_path TEXT, img_orig BYTEA, date DATE DEFAULT '1/1/1970' NOT NULL, face BOOLEAN DEFAULT 'False' NOT NULL, face_class INTEGER DEFAULT 0 NOT NULL)") # img_proc BYTEA, 
            cur.execute("COMMENT ON COLUMN images.id IS E'This number will come from a counter programatically.'")
            cur.execute("COMMENT ON COLUMN images.dir_path IS E'directory path to original image.  This is so that you can trace back to the imported data for files with the same names in multiple directories.'")
            cur.execute("COMMENT ON COLUMN images.date IS E'date image was imported.'")
            cur.execute("COMMENT ON COLUMN images.face_class IS E'Classes based off _# in filename.  0,1,2,3,4,5. 0=no_class, 1=commecicomm,2=smile,3=shut,4=shocked,5=sunglasses.'")
            cur.execute("ALTER TABLE images ADD CONSTRAINT dir_path PRIMARY KEY(id)")
            cur.execute("ALTER TABLE images ADD COLUMN class_pred INTEGER DEFAULT 0 NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN ext TEXT DEFAULT '.jpg' NOT NULL")
            con.commit()
            cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'images'")
            db_cols = cur.fetchall()
            print(db_cols)
        return 1
        

In [9]:
def bulk_insert_database(con, cur, id_, file_name, dir_path):

    with con:
        
        today = date.today().strftime('%m/%d/%Y')
        imgopen = open(dir_path, 'rb').read()
        face_class = file_name[-5]
        data = (id_, dir_path, psycopg2.Binary(imgopen), today , False, face_class)
        query = "INSERT INTO images(id, dir_path, img_orig, date, face, face_class) VALUES (%s, %s, %s, %s, %s, %s)" 
        cur.execute(query, data)
        
    return con, cur

In [10]:
def load_images_database(fname):
    file_name = '' # image name
    file_path = '' # path to file
    i = 0
    for file in fname:
        file_name = os.path.basename(file)
        file_path = file
        print('{}...{}...{}'.format(i, file_name, file_path))
        bulk_insert_database(con, cur, i, file_name, file_path) 
        i+=1
    return 1

In [11]:
# Update to take id and path as inputs.  id to select a specific row, path to where you wish to output (visualizations)
def write_image_database_to_file(con, cur):
    try:
        
        data = ('id', 'dir_path', 'date', 'img_orig' , 'face', 'face_class')
        cur.execute("SELECT data FROM images")
        data = cur.fetchone()
        open(os.path.expanduser('~/Desktop/test.jpp'), 'wb').write(data[3])        

    except psycopg2.DatabaseError as e:

        print('Error {}'.format(e))
        sys.exit(9)


In [12]:
def read_image_database(con):
    query = "SELECT id, dir_path, img_orig, face, face_class, class_pred FROM images"
    df_table = pd.read_sql_query(query, con)
    return df_table

In [13]:
def write_image_database(con, cur, data):
#     data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
#     cv2.imwrite('../test/test_image.jpg', data)
    return  7

# Preprocessing

In [14]:
! pwd

/Users/kennedy/Documents/GitHub/fiprojects/mod_capstone/notebooks


In [15]:
# # Renwming files as data fas the same filenames for each subject.  Prepending the folder name to the filename, as the foldername is the subject id.
# # ONLY RUN ONCE!!!!!!!
# for root, dirs, files in os.walk('../data/TD_RGB_E'):
#     if not files:
#         continue
#     prefix = os.path.basename(root)
#     for f in files:
#         os.rename(os.path.join(root, f), os.path.join(root, "{}_{}".format(prefix, f)))

In [62]:
# ONLY RUN ONCE
# sort_images('../data/base') 

1

In [16]:
file_name = '' # image name
file_path = '' # path to file

In [17]:
# Process Images Directory for Feeding into the Model
image_list = build_image_list('../data/TD_RGB_E')
print(len(image_list), image_list[0])

560 ../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_1.jpg


In [ ]:
# # copy files to a singular folder
# dest_base = '../data/base'
# for ig in image_list:
#     shutil.copy(ig, dest_base)

In [18]:
test_database()

PostgreSQL 12.1 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit


In [19]:
con, cur = connect_database()

In [20]:
create_database(con, cur)

[('id',), ('file_name',), ('dir_path',), ('img_orig',), ('date',), ('face',), ('face_class',), ('class_pred',), ('ext',)]


1

In [21]:
load_images_database(image_list)

0...95_TD_RGB_E_1.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_1.jpg
1...95_TD_RGB_E_3.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_3.jpg
2...95_TD_RGB_E_2.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_2.jpg
3...95_TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_5.jpg
4...95_TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_4.jpg
5...92_TD_RGB_E_3.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/92_TD_RGB_E_3.jpg
6...92_TD_RGB_E_2.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/92_TD_RGB_E_2.jpg
7...92_TD_RGB_E_1.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/92_TD_RGB_E_1.jpg
8...92_TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/92_TD_RGB_E_5.jpg
9...92_TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/92/92_TD_RGB_E_4.jpg
10...104_TD_RGB_E_4.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/104_TD_RGB_E_4.jpg
11...104_TD_RGB_E_5.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/104_TD_RGB_E_5.jpg
12...104_TD_RGB_E_2.jpg...../data/TD_RGB_E/TD_RGB_E_Set4/104/104_TD_RGB_E_2.jpg


1

In [22]:
# read from database
df = read_image_database(con)
df.head()

,id,dir_path,img_orig,face,face_class,class_pred
0,0,../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_...,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,1,0
1,1,../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_...,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,3,0
2,2,../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_...,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,2,0
3,3,../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_...,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,5,0
4,4,../data/TD_RGB_E/TD_RGB_E_Set4/95/95_TD_RGB_E_...,"[b'\xff', b'\xd8', b'\xff', b'\xe1', b'\xff', ...",False,4,0


In [ ]:
# from PIL import Image

# im = Image.open('temp.png')
# data = np.array(im)
# flattened = data.flatten()

# print data.shape
# print flattened.shape

# (612, 812, 4)
# (1987776,)

# Alternately, instead of calling data.flatten(), you could call data.reshape(-1). -1 is used as a placeholder for "figure out what the given dimension should be".

# flattened = data.T.flatten()

# TTS

In [ ]:
columns=['face_class']
y = df.face_class
X = df.drop(columns, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
print('X_train: {}'.format(len(X_train)))
print('y_train: {}'.format(len(y_train)))
print('X_test: {}'.format(len(X_test)))
print('y_test: {}'.format(len(y_test)))

In [ ]:
y_train.head()

# Crop Images with OpenCV

In [ ]:
for image in images:
    

In [ ]:
def detect_face(img_rgb, classifier):
    '''
    1st classifier: HAAR classifier for face detection
    read image from img_orig column
    process face
    if faces detected, update face column from False to True
    draw box around face
    write modified image to img_proc column
    call detect_eyes
    if no face detected, exit loop
    '''
    classifier_full = CascadeClassifier(classifier)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    bounding_boxes = classifier_full.detectMultiScale(img_gray, scaleFactor=1.3, minNeighbors=7, minSize=(224, 224))
    box_list = []
    for box in bounding_boxes:
        box_list.append(box)
    if len(box_list) < 1:
        return 0
    else:
        # update face to True
        draw_boxes(img_gray, box_list)
        draw_boxes(img_rgb, box_list)
        img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
        # write img to img_proc and/or save to os
    return img_gray, img_rgb, roi

In [ ]:
X_test.head()

# Main

In [ ]:
# set classifier
classifier = '../src/haarcascade_frontalface_default.xml'
# read the image
img_rgb = X_train['img_orig']

# detect face - with HAAR cascade
img_gray, img_rgb, roi = detect_face(img_rgb, classifier)

# scale down

# return roi

# call model for image - call fisherfaces

# write processed image to dataframe

# update flag in dataframe

# choose next image

# once complete, pueh changes to database


# Emotion Detection with VGG16 Pretrained CNN

### Convert the data into labels (encoded) and images converted to a 224x224x3 numpy array

In [29]:
emotions = ['commecicomm','happy','ugh','shocked','sunglasses']

In [77]:
!pwd

/Users/kennedy/Documents/GitHub/fiprojects/mod_capstone/notebooks


In [112]:
train_dir = '../data/train'
valid_dir = '../data/test'

train = ImageDataGenerator()
train_data = train.flow_from_directory(directory=train_dir,target_size=(224,224))
validation = ImageDataGenerator()
valid_data = validation.flow_from_directory(directory=valid_dir, target_size=(224,224))

Found 500 images belonging to 5 classes.
Found 60 images belonging to 5 classes.


In [113]:
image_size = 224
batch_size = 20


train_generator = train.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation.flow_from_directory(
                valid_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

Found 500 images belonging to 5 classes.
Found 60 images belonging to 5 classes.


In [99]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [100]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [101]:
base_model.trainable = False

In [115]:
model = Sequential([
    base_model,
    Dense(units=4096,activation="relu"),
    Dense(units=4096,activation="relu"),
    Dense(units=5, activation="softmax")
])

In [116]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [117]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
dense_4 (Dense)              (None, 7, 7, 4096)        2101248   
_________________________________________________________________
dense_5 (Dense)              (None, 7, 7, 4096)        16781312  
_________________________________________________________________
dense_6 (Dense)              (None, 7, 7, 5)           20485     
Total params: 33,617,733
Trainable params: 18,903,045
Non-trainable params: 14,714,688
_________________________________________________________________


In [118]:
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

Epoch 1/10


ValueError: Error when checking target: expected dense_6 to have 4 dimensions, but got array with shape (20, 1)

# Cleanup

In [ ]:
close_database(con)

# Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred_class)

tn = cm_2[0,0]
fn = cm_2[0,1]
fp = cm_2[1,0]
tp = cm_2[1,1]

accurracy = (tp + tn)/(tn+tp+fn+fp)

precision = tp / (tp+fp)

recall = tp / (tp + fn)
f1_score = 2*precision*recall/(precision+recall)
print('My model metrics were: Accurracy: {}, Precision: {}, Recall: {}, and F1: {}'.format(accurracy,precision,recall,f1_score))


plt.matshow(cm)

plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show();



# MVP

# Add webcam support

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, img = cap.read()
    '''
    lots of code here
    
    '''
    cv2.imshow('img', img)
    #Display camera feed until ESC key is pressed
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()